In [55]:
import requests
import pandas as pd
import numpy as np
import asyncio

In [ ]:
API_KEY = '' # подставить API ключ с сайта openweathermap.org
CITY = 'Moscow'
URL = f'http://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={API_KEY}&units=metric&lang=ru'

response = requests.get(URL)
data = response.json()

if response.status_code == 200:
    temp = data['main']['temp']
    descr = data['weather'][0]['description']
    print(f"Температура: {temp}°C, {descr}")
    print(data)
else:
    print("Ошибка:", data.get("message", "Неизвестная ошибка"))

Температура: -4.76°C, небольшой снег
{'coord': {'lon': 37.6156, 'lat': 55.7522}, 'weather': [{'id': 600, 'main': 'Snow', 'description': 'небольшой снег', 'icon': '13n'}], 'base': 'stations', 'main': {'temp': -4.76, 'feels_like': -8.87, 'temp_min': -4.76, 'temp_max': -4.71, 'pressure': 1011, 'humidity': 96, 'sea_level': 1011, 'grnd_level': 991}, 'visibility': 140, 'wind': {'speed': 2.68, 'deg': 231, 'gust': 8.65}, 'snow': {'1h': 0.2}, 'clouds': {'all': 83}, 'dt': 1767592449, 'sys': {'type': 1, 'id': 9027, 'country': 'RU', 'sunrise': 1767592661, 'sunset': 1767618693}, 'timezone': 10800, 'id': 524901, 'name': 'Москва', 'cod': 200}


In [12]:
df = pd.read_csv('temperature_data_aggregates.csv')
df.head()

,Unnamed: 0,city,season,average,standard_deviation
0,0,New York,winter,-0.246315,4.991234
1,0,New York,spring,9.950726,4.888135
2,0,New York,summer,24.997548,4.969963
3,0,New York,autumn,15.211042,5.014935
4,0,London,winter,4.727401,4.965666


In [23]:
def get_temperature_data(city):
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric&lang=ru'
    response = requests.get(url)
    data = response.json()

    if response.status_code == 200:
        temp = data['main']['temp']
        return temp
    else:
        return None


In [31]:
season = 'winter'
city = 'Sydney'

av = df[(df['season'] == season) & (df['city'] == city)]['average'].values[0].round(2)
st = df[(df['season'] == season) & (df['city'] == city)]['standard_deviation'].values[0].round(2)

print(f"Средняя температура в {season} в {city} составляет {av}°C, со стандартным отклонением {st}°C")
status = 'нормальная' if abs(get_temperature_data(city) - av) <= 2 * st else 'не нормальная'
print(f'Текущая температура в {city} составляет {get_temperature_data(city)}°C -- {status}')


Средняя температура в winter в Sydney составляет 11.75°C, со стандартным отклонением 4.98°C
Текущая температура в Sydney составляет 23.6°C -- не нормальная


Результат тестирование: для Москвы, Лондона, Нью-Йорка, Парижа и Дубая температура на момент 5 января 202 нормальная, а вот для Сиднея -- нет.

In [75]:
%%time
for i in range(10):
    response = requests.get(URL)

CPU times: user 41.6 ms, sys: 13.7 ms, total: 55.3 ms
Wall time: 4.04 s


In [83]:
import nest_asyncio
import aiohttp
nest_asyncio.apply()

async def fetch_weather():
    async with aiohttp.ClientSession() as session:
        async with session.get(URL) as resp:
            await resp.json()


In [84]:
import time
import asyncio

nest_asyncio.apply()

async def measure_time():
    start = time.time()
    tasks = [fetch_weather() for _ in range(10)]
    await asyncio.gather(*tasks)
    end = time.time()
    print(f"Время выполнения: {end - start:.2f} секунд")

await measure_time()


Время выполнения: 0.10 секунд
